In [1]:
import os

In [27]:
import pandas as pd
import numpy as np

In [75]:
fnames = [x for x in os.listdir() if x.endswith('csv')]

In [76]:
sorted(fnames, reverse=False)

['combined.csv',
 'config-blobs-1-results.csv',
 'config-blobs-2-results.csv',
 'config-blobs-3-results.csv',
 'config-xor-bad-10-2props-1w-results.csv',
 'config-xor-bad-11-2props-2w-results.csv',
 'config-xor-bad-6-results.csv',
 'config-xor-bad-7-results.csv',
 'config-xor-bad-8-samples-first-prop-2-weight-combs-results.csv']

In [80]:
dfs = {}
for name in fnames:
    dfs[name] = pd.read_csv(name)

In [78]:
dfs.keys()

dict_keys(['config-xor-bad-7-results.csv', 'config-xor-bad-6-results.csv', 'combined.csv', 'config-xor-bad-8-samples-first-prop-2-weight-combs-results.csv', 'config-blobs-3-results.csv', 'config-xor-bad-11-2props-2w-results.csv', 'config-xor-bad-10-2props-1w-results.csv', 'config-blobs-1-results.csv', 'config-blobs-2-results.csv'])

In [7]:
mask = dfs['config-xor-bad-6-results.csv']['time'] == 'timeout'
dfs['config-xor-bad-6-results.csv'][mask]

/usr/local/lib/python3.5/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,exp_id,avg_acc_before,avg_acc_after,time,Props,Heuristic,NumConstraints,Threshold,NNHidden,Params,Free,WeightsConfig,extra_params


In [40]:
from collections import OrderedDict
def agg_groups(df, key, skipped=False):
    res = []
    groups = df.groupby(key)
    for name, group in groups:
        print(name)
        _curr = {
            key: name,
            'N': len(group),
            'SoftConstraints': pd.to_numeric(group['NumConstraints'], errors='coerce').unique()[0],
            #'Thresholds': pd.to_numeric(group['Threshold'], errors='coerce').unique(),
            'AccBefore': pd.to_numeric(group['avg_acc_before'], errors='coerce').mean(),
            'MaxAcc': pd.to_numeric(group['avg_acc_after'], errors='coerce').max(),
            'MinAcc': pd.to_numeric(group['avg_acc_after'], errors='coerce').min(),
            'AvgAcc': pd.to_numeric(group['avg_acc_after'], errors='coerce').mean(),
            'SAT': pd.to_numeric(group['avg_acc_after'], errors='coerce').notnull().sum(),
            'UNSAT': (group['avg_acc_after']=='unsat').values.sum(),
            'Timeout': (group['time']=='timeout').values.sum(),
            'TotalSolverTime': int(pd.to_numeric(group['time'], errors='coerce').sum() + (group['time']=='timeout').values.sum()*600),
        }
        if skipped:
            _curr['Skipped'] = (group['time']=='skipped').values.sum()
        res.append(_curr)
    
    cols_order = ['N', 'SoftConstraints', key, 'AccBefore', 'MaxAcc', 'MinAcc', 'AvgAcc', 'SAT', 'UNSAT', 'Timeout', 'TotalSolverTime']
    if skipped:
        cols_order = cols_order[:-1] + ['Skipped'] + cols_order[-1:]
    df = pd.DataFrame(res)[cols_order]
    df = df.replace(np.nan, 'NA', regex=True)
    return df

In [41]:
float_format = "{0:.5%}"

### 1-1-weight comb-1st-property

In [42]:
a1 = agg_groups(dfs['config-xor-bad-6-results.csv'], 'Threshold', skipped=True)
a1

1
325
750
1000
1500
1561


/usr/local/lib/python3.5/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,N,SoftConstraints,Threshold,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,Skipped,TotalSolverTime
0,22,1562,1,0.995085,0.85964,0.375205,0.663572,9,13,0,0,110
1,22,1562,325,0.995085,0.842163,0.375205,0.646034,9,13,0,0,70
2,22,1562,750,0.995085,0.85964,0.495631,0.701734,8,14,0,0,69
3,22,1562,1000,0.995085,0.86947,0.842163,0.855816,4,18,0,0,57
4,22,1562,1500,NA,NA,NA,NA,0,22,0,0,58
5,22,1562,1561,NA,NA,NA,NA,0,22,0,0,34


In [43]:
print(a1.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{rrrllllrrrrr}
\toprule
  N &  SoftConstraints &  Threshold & AccBefore &    MaxAcc &    MinAcc &    AvgAcc &  SAT &  UNSAT &  Timeout &  Skipped &  TotalSolverTime \\
\midrule
 22 &             1562 &          1 & 99.50847\% & 85.96395\% & 37.52048\% & 66.35718\% &    9 &     13 &        0 &        0 &              110 \\
 22 &             1562 &        325 & 99.50847\% & 84.21628\% & 37.52048\% & 64.60343\% &    9 &     13 &        0 &        0 &               70 \\
 22 &             1562 &        750 & 99.50847\% & 85.96395\% & 49.56308\% & 70.17340\% &    8 &     14 &        0 &        0 &               69 \\
 22 &             1562 &       1000 & 99.50847\% & 86.94702\% & 84.21628\% & 85.58165\% &    4 &     18 &        0 &        0 &               57 \\
 22 &             1562 &       1500 &        NA &        NA &        NA &        NA &    0 &     22 &        0 &        0 &               58 \\
 22 &             1562 &       1561 &        NA &        NA &        NA 

In [153]:
#a1.to_csv('1-1-weight comb-1st-property.csv', index=False)

### 2-1-weight comb-2nd-property

In [46]:
a2 = agg_groups(dfs['config-xor-bad-7-results.csv'], 'Threshold', skipped=True)
a2

1
325
750
1000
1500
1561


/usr/local/lib/python3.5/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,N,SoftConstraints,Threshold,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,Skipped,TotalSolverTime
0,22,1562,1,0.995085,0.989896,0.380939,0.742354,10,12,0,0,141
1,22,1562,325,0.995085,0.979519,0.375205,0.741344,10,12,0,0,75
2,22,1562,750,0.995085,0.979519,0.498088,0.770647,9,13,0,0,63
3,22,1562,1000,0.995085,0.979519,0.640907,0.895481,8,14,0,0,68
4,22,1562,1500,0.995085,0.993173,0.963681,0.975014,4,18,0,0,64
5,22,1562,1561,NA,NA,NA,NA,0,22,0,0,27


In [47]:
print(a2.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{rrrllllrrrrr}
\toprule
  N &  SoftConstraints &  Threshold & AccBefore &    MaxAcc &    MinAcc &    AvgAcc &  SAT &  UNSAT &  Timeout &  Skipped &  TotalSolverTime \\
\midrule
 22 &             1562 &          1 & 99.50847\% & 98.98962\% & 38.09394\% & 74.23539\% &   10 &     12 &        0 &        0 &              141 \\
 22 &             1562 &        325 & 99.50847\% & 97.95194\% & 37.52048\% & 74.13435\% &   10 &     12 &        0 &        0 &               75 \\
 22 &             1562 &        750 & 99.50847\% & 97.95194\% & 49.80885\% & 77.06475\% &    9 &     13 &        0 &        0 &               63 \\
 22 &             1562 &       1000 & 99.50847\% & 97.95194\% & 64.09066\% & 89.54806\% &    8 &     14 &        0 &        0 &               68 \\
 22 &             1562 &       1500 & 99.50847\% & 99.31731\% & 96.36810\% & 97.50137\% &    4 &     18 &        0 &        0 &               64 \\
 22 &             1562 &       1561 &        NA &        NA &       

In [156]:
#a2.to_csv('2-1-weight comb-2nd-property.csv', index=False)

### 3-2-weight comb-1st-property

In [50]:
a3 = agg_groups(dfs['config-xor-bad-8-samples-first-prop-2-weight-combs-results.csv'], 'Threshold', skipped=True)
a3['SoftConstraints'] = int(1562)
a3['Threshold'] = a3['Threshold'].astype('int32')
a3

1.0
325.0
750.0
1000.0
1500.0


,N,SoftConstraints,Threshold,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,Skipped,TotalSolverTime
0,231,1562,1,0.995085,0.992081,0.337247,0.659105,156,61,14,0,9127
1,231,1562,325,0.995085,0.991262,0.377389,0.680571,103,0,53,75,42027
2,231,1562,750,0.995085,0.992627,0.489077,0.713595,92,0,11,128,18004
3,230,1562,1000,0.995085,0.986073,0.612234,0.805563,74,11,6,139,13321
4,230,1562,1500,0.995085,0.992627,0.931185,0.969689,18,51,5,156,9529


In [51]:
print(a3.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{rrrrrrrrrrrr}
\toprule
   N &  SoftConstraints &  Threshold &  AccBefore &    MaxAcc &    MinAcc &    AvgAcc &  SAT &  UNSAT &  Timeout &  Skipped &  TotalSolverTime \\
\midrule
 231 &             1562 &          1 &  99.50847\% & 99.20808\% & 33.72474\% & 65.91046\% &  156 &     61 &       14 &        0 &             9127 \\
 231 &             1562 &        325 &  99.50847\% & 99.12616\% & 37.73894\% & 68.05714\% &  103 &      0 &       53 &       75 &            42027 \\
 231 &             1562 &        750 &  99.50847\% & 99.26270\% & 48.90770\% & 71.35950\% &   92 &      0 &       11 &      128 &            18004 \\
 230 &             1562 &       1000 &  99.50847\% & 98.60732\% & 61.22338\% & 80.55633\% &   74 &     11 &        6 &      139 &            13321 \\
 230 &             1562 &       1500 &  99.50847\% & 99.26270\% & 93.11851\% & 96.96887\% &   18 &     51 &        5 &      156 &             9529 \\
\bottomrule
\end{tabular}



In [ ]:
a4

In [160]:
#a3.to_csv('3-Selecting multiple weights-neurons-or-layers-2nd-property.csv', index=False)

#### 4-2-props-1-weight-combs

In [53]:
a4 = agg_groups(dfs['config-xor-bad-10-2props-1w-results.csv'], 'Threshold', skipped=True)
a4['SoftConstraints'] = int(1562)
a4

1
325
750
1000
1500
1561


,N,SoftConstraints,Threshold,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,Skipped,TotalSolverTime
0,22,1562,1,0.995085,0.842163,0.375205,0.61065,5,17,0,0,119
1,22,1562,325,0.995085,0.842163,0.375205,0.610431,5,0,0,17,31
2,22,1562,750,0.995085,0.842163,0.494539,0.669238,4,1,0,17,27
3,22,1562,1000,0.995085,0.842163,0.842163,0.842163,2,2,0,18,16
4,22,1562,1500,NA,NA,NA,NA,0,2,0,20,7
5,22,1562,1561,NA,NA,NA,NA,0,0,0,22,0


In [54]:
print(a4.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{rrrllllrrrrr}
\toprule
  N &  SoftConstraints &  Threshold & AccBefore &    MaxAcc &    MinAcc &    AvgAcc &  SAT &  UNSAT &  Timeout &  Skipped &  TotalSolverTime \\
\midrule
 22 &             1562 &          1 & 99.50847\% & 84.21628\% & 37.52048\% & 61.06499\% &    5 &     17 &        0 &        0 &              119 \\
 22 &             1562 &        325 & 99.50847\% & 84.21628\% & 37.52048\% & 61.04315\% &    5 &      0 &        0 &       17 &               31 \\
 22 &             1562 &        750 & 99.50847\% & 84.21628\% & 49.45385\% & 66.92381\% &    4 &      1 &        0 &       17 &               27 \\
 22 &             1562 &       1000 & 99.50847\% & 84.21628\% & 84.21628\% & 84.21628\% &    2 &      2 &        0 &       18 &               16 \\
 22 &             1562 &       1500 &        NA &        NA &        NA &        NA &    0 &      2 &        0 &       20 &                7 \\
 22 &             1562 &       1561 &        NA &        NA &        NA 

### 5-2props-2-weights

In [58]:
a5 = agg_groups(dfs['config-xor-bad-11-2props-2w-results.csv'], 'Threshold', skipped=True)
a5['SoftConstraints'] = int(1562)
a5

1
325
750
1000
1500


,N,SoftConstraints,Threshold,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,Skipped,TotalSolverTime
0,231,1562,1,0.995085,0.972146,0.200437,0.577172,119,95,14,0,8933
1,231,1562,325,0.995085,0.904424,0.361551,0.637946,76,0,43,112,34731
2,231,1562,750,0.995085,0.973512,0.489077,0.682485,65,0,11,155,11555
3,231,1562,1000,0.995085,0.904424,0.636264,0.809612,40,16,9,166,11202
4,231,1562,1500,0.995085,0.970508,0.962862,0.966685,2,37,1,191,3676


In [59]:
print(a5.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{rrrrrrrrrrrr}
\toprule
   N &  SoftConstraints &  Threshold &  AccBefore &    MaxAcc &    MinAcc &    AvgAcc &  SAT &  UNSAT &  Timeout &  Skipped &  TotalSolverTime \\
\midrule
 231 &             1562 &          1 &  99.50847\% & 97.21464\% & 20.04369\% & 57.71723\% &  119 &     95 &       14 &        0 &             8933 \\
 231 &             1562 &        325 &  99.50847\% & 90.44238\% & 36.15511\% & 63.79459\% &   76 &      0 &       43 &      112 &            34731 \\
 231 &             1562 &        750 &  99.50847\% & 97.35117\% & 48.90770\% & 68.24854\% &   65 &      0 &       11 &      155 &            11555 \\
 231 &             1562 &       1000 &  99.50847\% & 90.44238\% & 63.62643\% & 80.96122\% &   40 &     16 &        9 &      166 &            11202 \\
 231 &             1562 &       1500 &  99.50847\% & 97.05079\% & 96.28618\% & 96.66849\% &    2 &     37 &        1 &      191 &             3676 \\
\bottomrule
\end{tabular}



### Blobs

### 1st prop - 1 weight comb

In [81]:
dfs['config-blobs-1-results.csv'].shape

(535, 14)

In [82]:
a6 = agg_groups(dfs['config-blobs-1-results.csv'], 'Threshold', skipped=True)
a6['SoftConstraints'] = int(6000)
a6

1
500
1000
2000
5500


,N,SoftConstraints,Threshold,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,Skipped,TotalSolverTime
0,107,6000,1,1.0,0.994364,0.641000,0.895250,48,44,15,0,18968
1,107,6000,500,1.0,0.994182,0.865091,0.950344,32,0,16,59,11723
2,107,6000,1000,1.0,0.995000,0.865091,0.941142,32,0,0,75,2046
3,107,6000,2000,1.0,0.994364,0.864727,0.944440,32,0,0,75,2330
4,107,6000,5500,1.0,0.993455,0.904909,0.955502,29,3,0,75,3518


In [83]:
print(a6.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{rrrrrrrrrrrr}
\toprule
   N &  SoftConstraints &  Threshold &  AccBefore &    MaxAcc &    MinAcc &    AvgAcc &  SAT &  UNSAT &  Timeout &  Skipped &  TotalSolverTime \\
\midrule
 107 &             6000 &          1 & 100.00000\% & 99.43636\% & 64.10000\% & 89.52500\% &   48 &     44 &       15 &        0 &            18968 \\
 107 &             6000 &        500 & 100.00000\% & 99.41818\% & 86.50909\% & 95.03438\% &   32 &      0 &       16 &       59 &            11723 \\
 107 &             6000 &       1000 & 100.00000\% & 99.50000\% & 86.50909\% & 94.11420\% &   32 &      0 &        0 &       75 &             2046 \\
 107 &             6000 &       2000 & 100.00000\% & 99.43636\% & 86.47273\% & 94.44403\% &   32 &      0 &        0 &       75 &             2330 \\
 107 &             6000 &       5500 & 100.00000\% & 99.34545\% & 90.49091\% & 95.55016\% &   29 &      3 &        0 &       75 &             3518 \\
\bottomrule
\end{tabular}



###  2nd prop - 1 w comb

In [84]:
a7 = agg_groups(dfs['config-blobs-2-results.csv'], 'Threshold', skipped=True)
a7['SoftConstraints'] = int(6000)
a7

1
500
1000
2000
5500


,N,SoftConstraints,Threshold,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,Skipped,TotalSolverTime
0,143,6000,1,1.0,0.997273,0.511273,0.866613,31,95,17,0,23884
1,143,6000,500,1.0,0.997273,0.861545,0.927232,18,0,13,112,9573
2,143,6000,1000,1.0,0.997273,0.863091,0.935980,18,0,0,125,1406
3,143,6000,2000,1.0,0.997545,0.646727,0.905212,18,0,0,125,1789
4,143,6000,5500,1.0,0.996364,0.904727,0.945016,17,1,0,125,2580


In [86]:
print(a7.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{rrrrrrrrrrrr}
\toprule
   N &  SoftConstraints &  Threshold &  AccBefore &    MaxAcc &    MinAcc &    AvgAcc &  SAT &  UNSAT &  Timeout &  Skipped &  TotalSolverTime \\
\midrule
 143 &             6000 &          1 & 100.00000\% & 99.72727\% & 51.12727\% & 86.66129\% &   31 &     95 &       17 &        0 &            23884 \\
 143 &             6000 &        500 & 100.00000\% & 99.72727\% & 86.15455\% & 92.72323\% &   18 &      0 &       13 &      112 &             9573 \\
 143 &             6000 &       1000 & 100.00000\% & 99.72727\% & 86.30909\% & 93.59798\% &   18 &      0 &        0 &      125 &             1406 \\
 143 &             6000 &       2000 & 100.00000\% & 99.75455\% & 64.67273\% & 90.52121\% &   18 &      0 &        0 &      125 &             1789 \\
 143 &             6000 &       5500 & 100.00000\% & 99.63636\% & 90.47273\% & 94.50160\% &   17 &      1 &        0 &      125 &             2580 \\
\bottomrule
\end{tabular}



In [87]:
a8 = agg_groups(dfs['config-blobs-3-results.csv'], 'Threshold', skipped=True)
a8['SoftConstraints'] = int(6000)
a8

1
500
1000
2000
5500


,N,SoftConstraints,Threshold,AccBefore,MaxAcc,MinAcc,AvgAcc,SAT,UNSAT,Timeout,Skipped,TotalSolverTime
0,45,6000,1,1.0,0.994909,0.624545,0.891556,33,12,0,0,5137
1,45,6000,500,1.0,0.991091,0.741636,0.932626,9,0,24,12,15823
2,45,6000,1000,1.0,0.995636,0.863091,0.930864,4,0,5,36,3477
3,45,6000,2000,1.0,0.868909,0.863091,0.865545,3,0,1,41,747
4,45,6000,5500,1.0,0.988182,0.973545,0.982364,3,0,0,42,638


In [88]:
print(a8.to_latex(index=False, float_format=float_format.format))

\begin{tabular}{rrrrrrrrrrrr}
\toprule
  N &  SoftConstraints &  Threshold &  AccBefore &    MaxAcc &    MinAcc &    AvgAcc &  SAT &  UNSAT &  Timeout &  Skipped &  TotalSolverTime \\
\midrule
 45 &             6000 &          1 & 100.00000\% & 99.49091\% & 62.45455\% & 89.15565\% &   33 &     12 &        0 &        0 &             5137 \\
 45 &             6000 &        500 & 100.00000\% & 99.10909\% & 74.16364\% & 93.26263\% &    9 &      0 &       24 &       12 &            15823 \\
 45 &             6000 &       1000 & 100.00000\% & 99.56364\% & 86.30909\% & 93.08636\% &    4 &      0 &        5 &       36 &             3477 \\
 45 &             6000 &       2000 & 100.00000\% & 86.89091\% & 86.30909\% & 86.55455\% &    3 &      0 &        1 &       41 &              747 \\
 45 &             6000 &       5500 & 100.00000\% & 98.81818\% & 97.35455\% & 98.23636\% &    3 &      0 &        0 &       42 &              638 \\
\bottomrule
\end{tabular}



### Combined

In [206]:
for name, df in dfs.items():
    df['source'] = name.split('.')[0]
    alldfs = alldfs.append(df)

In [207]:
alldfs = alldfs.set_index(['source', 'exp_id'])

In [208]:
alldfs.head()

avg_acc_before avg_acc_after     time  Props  \
source                   exp_id                                                
config-xor-bad-1-results 1            0.995085      0.650464   0.2746      1   
                         2            0.995085      0.981704  30.4846      1   
                         3            0.995085      0.981704   3.0699      1   
                         4            0.995085      0.649918   0.3767      1   
                         5            0.995085      0.979792   4.3493      1   

                                Heuristic  NumConstraints  Threshold NNHidden  \
source                   exp_id                                                 
config-xor-bad-1-results 1        SAMPLES             450          1      [4]   
                         2        SAMPLES             450        250      [4]   
                         3        SAMPLES             450        400      [4]   
                         4        SAMPLES             450          1      [4]   
                         5        SAMPLES             450        250      [4]   

                                 Params  Free                WeightsConfig  \
source                   exp_id                                              
config-xor-bad-1-results 1           22     5  {'select_neuron': [[2, 1]]}   
                         2           22     5  {'select_neuron': [[2, 1]]}   
                         3           22     5  {'select_neuron': [[2, 1]]}   
                         4           22     5  {'select_neuron': [[2, 2]]}   
                         5           22     5  {'select_neuron': [[2, 2]]}   

                                extra_params  
source                   exp_id               
config-xor-bad-1-results 1               NaN  
                         2               NaN  
                         3               NaN  
                         4               NaN  
                         5               NaN

In [210]:
alldfs.to_csv('combined.csv')